In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !pip install numpy==1.16.1
# !pip install tensorpack

!pip install numpy==1.15.4
!pip install tensorpack==0.9.8
!pip install tensorflow==1.14
!pip install tensorflow-gpu==1.14
!pip install gast==0.2.2

     |████████████████████████████████| 13.9MB 2.8MB/s 
ERROR: tensorflow 1.15.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.3
    Uninstalling numpy-1.17.3:
      Successfully uninstalled numpy-1.17.3


     |████████████████████████████████| 296kB 2.8MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


     |████████████████████████████████| 109.2MB 69kB/s 
     |████████████████████████████████| 3.2MB 32.4MB/s 
     |████████████████████████████████| 491kB 41.6MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 377.0MB 36kB/s 


In [0]:
import numpy as np
import argparse
import os
import cv2
import sys
from glob import glob

from tensorpack import *
from tensorpack.tfutils.sessinit import get_model_loader
from tensorpack.tfutils.symbolic_functions import *
from tensorpack.tfutils.summary import *
from tensorpack.utils.gpu import get_nr_gpu
from tensorpack.dataflow.base import RNGDataFlow
from tensorpack.tfutils.tower import get_current_tower_context
import tensorflow as tf

In [0]:
class Scene15(RNGDataFlow):

    def __init__(self, dir, name, img_size, meta_dir=None,
                 shuffle=None, dir_structure=None):

        assert name in ['train', 'test'], name
        assert os.path.isdir(dir), dir
        self.full_dir = os.path.join(dir, name)
        self.name = name
        assert os.path.isdir(self.full_dir), self.full_dir
        if shuffle is None:
            shuffle = name == 'train'
        self.shuffle = shuffle

        # For each category, add up to the self-enforced limit on the number of training/test examples
        #
        self.imglist = []
        for catname in glob('%s/%s/*' % (dir, name)):
            catlist = glob('%s/*' % catname)
            c = 0
            for fname in catlist:
                self.imglist.append( (fname, os.path.basename(os.path.dirname(fname))) )
                c = c+1
                if name == 'train' and c >= 100:
                    break
                if name == 'test' and c >= 100:
                    break

        # Compact variant with no limits; just read all the data
        # We don't do this for speed reasons
        # self.imglist2 = [(fname, os.path.basename(os.path.dirname(fname))) for fname in glob('%s/%s/*/*' % (dir, name))]

        self.label_lookup = dict()
        for label in sorted(set(i[1] for i in self.imglist)):
            self.label_lookup[label] = len(self.label_lookup)

        self.imglist = [(fname, self.label_lookup[dirname]) for fname, dirname in self.imglist]

        
        idxs = np.arange(len(self.imglist))

        # Load images into numpy array
        self.imgs = np.zeros( (img_size, img_size, 3, len(self.imglist) ), dtype=np.float )
        for k in idxs:
            fname, label = self.imglist[k]
            img = cv2.resize( cv2.imread(fname), (img_size, img_size) )
            img = img / 255.0 # You might want to remove this line for your standardization.
            self.imgs[:,:,:,k] = img

        ########################################################
        # TASK 1: Add standardization (also called feature scaling or data normalization).
        # Reference here: https://en.wikipedia.org/wiki/Feature_scaling
        # To think about: In what different ways could we 'standardize' our data? What would each do?
        
        mean_img = np.mean(self.imgs, axis = 3)
        std_img = np.std(self.imgs, axis = 3)

        for k in idxs:
            self.imgs[:,:,:,k] = (self.imgs[:,:,:,k] - mean_img) / std_img

        ########################################################


    def size(self):
        return len(self.imglist)

    def get_data(self):
        idxs = np.arange(len(self.imglist))
        if self.shuffle:
            self.rng.shuffle(idxs)
        for k in idxs:
            fname, label = self.imglist[k]
            fname = os.path.join(self.full_dir, fname)
            yield [self.imgs[:,:,:,k], label]

In [0]:
"""
Convenience function to load the 15 Scene database.
This is where you would place any potential data augmentations.
"""
def get_data(datadir, train_or_test):
    isTrain = train_or_test == 'train'
    img_size = 224 # Hard coded, as VGG-16 network must have this input size

    ds = Scene15(datadir, train_or_test, img_size)
    if isTrain:
        augmentors = [
            #################################################
            # TASK 1: Add data augmentations
            #
            # An example (that is duplicated work).
            # In the Scene15 class, we resize each image to 
            # 64x64 pixels as a preprocess. You then perform
            # standardization over the images in Task 1.
            #
            # However, if we wanted to skip standardization, 
            # we could use an augmentation to resize the image
            # whenever it is needed:
            # imgaug.Resize( (img_size, img_size) )
            #
            # Please use the same syntax to write more useful 
            # augmentations. Read the documentation on the 
            # TensorPack image augmentation library and experiment!
            #################################################
            # imgaug.RandomCrop((50, 50)),
            # imgaug.Resize((img_size, img_size)),
            # imgaug.Shift()
        ]
    else:
        # Validation/test time augmentations
        augmentors = [
            # imgaug.Resize( (img_size, img_size) ) 
            # imgaug.RandomCrop((50, 50)),
            # imgaug.Resize((img_size, img_size)),
            # imgaug.Shift()
        ]
    # TensorPack: Add data augmentations
    ds = AugmentImageComponent(ds, augmentors)
    # TensorPack: How to batch the data
    # if isTrain:
    #     with open('train_data', 'rb') as train_data_file:
    #         ds = pickle.load(train_data_file)
    # else:
    #   with open('test_data', 'rb') as test_data_file:
    #         ds = pickle.load(test_data_file)
    ds = BatchData(ds, 50, remainder=not isTrain)
    if isTrain:
        # TensorPack: Perform clever image fetching, e.g., multithreaded
        # These numbers will depend on your particular machine.
        # Note: PrefetchData seems to be broken on Windows : /
        if not sys.platform.lower().startswith('win'):
            ds = PrefetchData(ds, 4, 2)
    return ds

In [0]:
def prediction_incorrect(logits, label, topk=1, name='incorrect_vector'):
  with tf.name_scope('prediction_incorrect'):
    # x = tf.logical_not(tf.nn.in_top_k(label, logits, topk))
    x = tf.logical_not(tf.nn.in_top_k(logits, label, topk))
  return tf.cast(x, tf.float32, name=name)


class VGGModel(ModelDesc):
  def __init__(self):
    super(VGGModel, self).__init__()
    self.activation_fn = tf.nn.relu
    self.conv_padding = 'SAME'
    self.pool_padding = 'SAME'
    self.use_bias = True


  def inputs(self):
    return [tf.TensorSpec([None, 224, 224, 3], tf.float32, 'input'),
            tf.TensorSpec([None], tf.int32, 'label')]

  def build_graph(self, image, label):
    ################################################################################
    # TASK 2: Fine tuning
    #
    # We wish to replace VGG's last fully connected layer. 
    # We need to change the number of classes for our output, too.
    #
    # Each layer has a name. TensorPack will load the weights from vgg16.npy and 
    # match the names of the specified layers to the layers which exist in vgg16.npy.
    # In this case, we need to _use a different name than fc8_, otherwise TensorPack
    # will copy over the existing weights.
    #
    # Training will take _a long time_ when not on a GPU - two hours per epoch on 
    # James' laptop CPU. Run it using Google Cloud Platform!
    # There is also a feature to continue from where you left off (per epoch).
    # You'll notice it once you've executed run.py multiple times.
    #
    # Weight freezing: It is also possible to stop gradient propagation beyond the 
    # newly added fc layer. This will limit the ability of the pre-trained network to 
    # adjust to the new data, but if the representation is already sufficient then it
    # will converge much faster. Please investigate this via tf.stop_gradient()
    #
    ################################################################################

    # with tf.variable_scope('var_scope', reuse=tf.AUTO_REUSE):
    with argscope(Conv2D, kernel_shape=3, nl=tf.nn.relu):
      logits = (LinearWrap(image)
                .Conv2D('conv1_1', 64)
                .Conv2D('conv1_2', 64)
                .MaxPooling('pool1', 2)
                # 112
                .Conv2D('conv2_1', 128)
                .Conv2D('conv2_2', 128)
                .MaxPooling('pool2', 2)
                # 56
                .Conv2D('conv3_1', 256)
                .Conv2D('conv3_2', 256)
                .Conv2D('conv3_3', 256)
                .MaxPooling('pool3', 2)
                # 28
                .Conv2D('conv4_1', 512)
                .Conv2D('conv4_2', 512)
                .Conv2D('conv4_3', 512)
                .MaxPooling('pool4', 2)
                # 14
                .Conv2D('conv5_1', 512)
                .Conv2D('conv5_2', 512)
                .Conv2D('conv5_3', 512)
                .MaxPooling('pool5', 2)
                # 7
                .FullyConnected('fc6', 4096, nl=tf.nn.relu)
                .FullyConnected('fc7', 4096, nl=tf.nn.relu)
                # .FullyConnected('fc8', out_dim=1000, nl=tf.identity)()

                .FullyConnected('fc11', 4096, nl=tf.nn.relu)
                .FullyConnected('fc12', out_dim=15, nl=tf.identity)()
                )
          
    prob = tf.nn.softmax(logits, name='output')

    cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=label)
    cost = tf.reduce_mean(cost, name='cross_entropy_loss')

    wrong = prediction_incorrect(logits, label)

    # monitor training error
    add_moving_summary(tf.reduce_mean(wrong, name='train_error'))

    add_moving_summary(cost)

    add_param_summary(('.*/W', ['histogram']))   # monitor W
    self.cost = tf.add_n([cost], name='cost')
    return self.cost

  def optimizer(self):
        # opt = tf.compat.v1.train.RMSPropOptimizer(hp.learning_rate)
        opt = tf.train.RMSPropOptimizer(0.01)
        return opt

In [0]:
# parser = argparse.ArgumentParser()
# parser.add_argument(
#     '--task',
#     required=True,
#     choices=['1', '2'],
#     help='Which task of the assignment to run - training from scratch (1), or fine tuning VGG-16 (2).')
# # Set GPU to -1 to not use a GPU.
# parser.add_argument('--gpu', help='Comma-separated list of GPU(s) to use.')
# parser.add_argument(
#     '--load',
#     # Location of pre-trained model
#     # - As a relative path to the student distribution
#     default='../vgg16.npy',
#     # - As an absolute path to the location on the Brown CS filesystem
#     #default='/course/cs1430/pretrained_weights/vgg16.npy',
#     help='Load VGG-16 model.')
load = '/content/drive/My Drive/vgg16.npy'
# parser.add_argument(
#     '--data',
#     # Location of 15 Scenes dataset
#     # - As a relative path to the student distribution
#     default=os.getcwd() + '/../data/',
#     # - As an absolute path to the location on the Brown CS filesystem
#     #default='/course/cs1430/datasets/15SceneData/',
#     help='Location where the dataset is stored.')
data = '/content/drive/My Drive/data/'

# args = parser.parse_args()

# if args.gpu:
#     os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

# logger.auto_set_dir()
logger.set_logger_dir('/content/drive/My Drive/tensorpack_log')

dataset_train = get_data(data, 'train')
dataset_test = get_data(data, 'test')

[1112 17:30:55 @logger.py:126] WRN Log directory /content/drive/My Drive/tensorpack_log exists! Use 'd' to delete it. 
[1112 17:30:55 @logger.py:129] WRN If you're resuming from a previous run, you can choose to keep it.
Press any other key to exit. 
Select Action: k (keep) / d (delete) / q (quit):k
[1112 17:30:58 @logger.py:90] Argv: /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-0aa7bfb5-418b-4f98-a12e-561ed97477bd.json
[1112 17:40:09 @parallel.py:231] [MultiProcessRunner] Will fork a dataflow more than one times. This assumes the datapoints are i.i.d.
[1112 17:40:09 @argtools.py:146] WRN Starting a process with 'fork' method is not safe and may consume unnecessary extra CPU memory. Use 'forkserver/spawn' method (available after Py3.4) instead if you run into any issues. See https://docs.python.org/3/library/multiprocessing.html#contexts-and-start-methods
[1112 17:40:09 @argtools.py:146] WRN "import prctl" failed! Install pyt

In [0]:
logger.set_logger_dir('/content/drive/My Drive/tensorpack_log')

[1112 16:31:53 @logger.py:126] WRN Log directory /content/drive/My Drive/tensorpack_log exists! Use 'd' to delete it. 
[1112 16:31:53 @logger.py:129] WRN If you're resuming from a previous run, you can choose to keep it.
Press any other key to exit. 
Select Action: k (keep) / d (delete) / q (quit):d
[1112 16:31:55 @logger.py:90] Argv: /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-bece6984-1673-4679-9ae7-02c07c5c2fb0.json


In [0]:
# TensorPack: Training configuration
config = TrainConfig(
    model=VGGModel(),
    dataflow=dataset_train,
    callbacks=[
        # Callbacks are performed at the end of every epoch.
        #
        # For instance, we can save the current model
        ModelSaver(),
        # Evaluate the current model and print out the loss
        InferenceRunner(dataset_test,
                        [ScalarStats('cost'), ClassificationError()])
        #
        # You can put other callbacks here to change hyperparameters,
        # etc...
        #
    ],
    max_epoch=30,
    nr_tower=max(get_nr_gpu(), 1),
    session_init=get_model_loader(load)
)
launch_train_with_config(config, SimpleTrainer())

[1112 17:49:58 @sessinit.py:294] Loading dictionary from /content/drive/My Drive/vgg16.npy ...
[1112 17:50:03 @trainers.py:47] Building graph for a single training tower ...
[1112 17:50:04 @registry.py:90] 'conv1_1': [?, 224, 224, 3] --> [?, 224, 224, 64]
[1112 17:50:04 @registry.py:90] 'conv1_2': [?, 224, 224, 64] --> [?, 224, 224, 64]
[1112 17:50:04 @registry.py:90] 'pool1': [?, 224, 224, 64] --> [?, 112, 112, 64]
[1112 17:50:04 @registry.py:90] 'conv2_1': [?, 112, 112, 64] --> [?, 112, 112, 128]
[1112 17:50:04 @registry.py:90] 'conv2_2': [?, 112, 112, 128] --> [?, 112, 112, 128]
[1112 17:50:04 @registry.py:90] 'pool2': [?, 112, 112, 128] --> [?, 56, 56, 128]
[1112 17:50:04 @registry.py:90] 'conv3_1': [?, 56, 56, 128] --> [?, 56, 56, 256]
[1112 17:50:04 @registry.py:90] 'conv3_2': [?, 56, 56, 256] --> [?, 56, 56, 256]
[1112 17:50:04 @registry.py:90] 'conv3_3': [?, 56, 56, 256] --> [?, 56, 56, 256]
[1112 17:50:04 @registry.py:90] 'pool3': [?, 56, 56, 256] --> [?, 28, 28, 256]
[1112 17

100%|##########|30/30[00:25<00:00, 1.16it/s]

[1112 17:50:41 @base.py:285] Epoch 1 (global_step 30) finished, time:25.8 seconds.


[1112 17:50:50 @saver.py:82] Model saved to /content/drive/My Drive/tensorpack_log/model-30.


100%|##########|30/30[00:09<00:00, 3.09it/s]

...

[1112 17:58:58 @base.py:285] Epoch 16 (global_step 480) finished, time:16.5 seconds.


[1112 17:59:05 @saver.py:82] Model saved to /content/drive/My Drive/tensorpack_log/model-480.


100%|##########|30/30[00:09<00:00, 3.18it/s]

[1112 17:59:14 @monitor.py:474] cross_entropy_loss: 5.197e+06
[1112 17:59:14 @monitor.py:474] train_error: 0.92903
[1112 17:59:14 @monitor.py:474] validation_cost: 2.9458
[1112 17:59:14 @monitor.py:474] validation_error: 0.93333
[1112 17:59:14 @group.py:48] Callbacks took 16.141 sec in total. ModelSaver: 6.7 seconds; InferenceRunner: 9.43 seconds
[1112 17:59:14 @base.py:275] Start Epoch 17 ...



 20%|##        |6/30[00:02<00:08, 2.76it/s]